#### LUT→RIPPER Truth‑Table Distillation | 100‑bit Artificial
# Replace each LUT node with a RIPPER node trained **only on its own truth table**


### 0 · Imports

In [1]:
import itertools, numpy as np, pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from architecture_goodie.deep_binary_classifier import DeepBinaryClassifier
from architecture_goodie.lut_node      import make_lut_node
from architecture_goodie.ripper_node   import make_ripper_node


### 1 · Load dataset

In [2]:
df = pd.read_csv('./data/100_bit_artificial/1a.csv')
X = df.drop(columns='class').to_numpy(dtype=bool)
y = df['class'].to_numpy(dtype=bool)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Dataset shape              : {df.shape}')
print(f'Train/Test split           : {X_train.shape[0]} / {X_test.shape[0]}')
print(f'Train label dist (T/F)     : {y_train.sum()} / {y_train.size - y_train.sum()}')


Dataset shape              : (10000, 101)
Train/Test split           : 8000 / 2000
Train label dist (T/F)     : 4605 / 3395


### 2 · Train baseline LUT network

In [3]:
layer_count, node_count, bit_count = 4, 32, 4

lut_net = DeepBinaryClassifier(
    nodes_per_layer   =[node_count]*layer_count,
    bits_per_node     =[bit_count]*(layer_count+1),
    node_factory      =make_lut_node,
    rng               =42,
    n_jobs            =1,
    reuse_prev_width  =True,
).fit(X_train, y_train)

acc_lut = accuracy_score(y_test, lut_net.predict(X_test))
print(f'Baseline LUT network accuracy: {acc_lut:.4f}')


Baseline LUT network accuracy: 0.7445


### 3 · Distil each node via its **own** truth table

* Build complete Boolean truth‑table for that node’s `bits`.  
* Label rows with the node’s current LUT output.  
* Train a `RipperNode` on this synthetic dataset.  
* Replace the original node object in the network.


In [4]:
def distil_node_with_ripper(node, *, seed=0):
    """Return a RipperNode that mimics **exactly** the given LutNode."""
    bits, cols = node.bits, node.cols
    # 2^bits patterns
    patterns = np.array(list(itertools.product([False, True], repeat=bits)), dtype=bool)
    y_bool   = node.lut.copy()
    y_pm1    = y_bool.astype(np.int8)*2 - 1

    rip_node = make_ripper_node(
        patterns, y_pm1, bits, cols,
        rng            = np.random.default_rng(seed),
        tie_break      = 'zero',                 # no ties exist
        ripper_kwargs  = {'random_state': seed}
    )
    return rip_node

### 4 · Swap every LUT‑node for its distilled RIPPER clone

In [5]:
def distil_network(lut_model, *, seed=0):
    for layer_idx, layer in enumerate(lut_model.layers):
        for i, node in enumerate(layer):
            if not hasattr(node, 'lut'):      # already a RipperNode?
                continue
            rip = distil_node_with_ripper(node, seed=seed)
            lut_model.layers[layer_idx][i] = rip
    return lut_model

ripped_net = distil_network(lut_net, seed=0)
acc_ripped = accuracy_score(y_test, ripped_net.predict(X_test))
print(f'Accuracy after truth‑table distillation: {acc_ripped:.4f}')


preprocess.py.py: ._get_pos_class: RuntimeWarning: 
No negative samples. All target labels=True.

ripper.py: .fit: RuntimeWarning: 
No negative samples. Existing target labels=[True].

ripper.py: .fit | base.py: ._check_allpos_allneg: RuntimeWarning: 
Ruleset is empty. All predictions it makes with method .predict will be negative. It may be untrained or was trained on a dataset split lacking positive examples.

base.py: .predict | base.py: ._check_allpos_allneg: RuntimeWarning: 
Ruleset is empty. All predictions it makes with method .predict will be negative. It may be untrained or was trained on a dataset split lacking positive examples.

preprocess.py.py: ._get_pos_class: RuntimeWarning: 
No negative samples. All target labels=True.

ripper.py: .fit: RuntimeWarning: 
No negative samples. Existing target labels=[True].

ripper.py: .fit | base.py: ._check_allpos_allneg: RuntimeWarning: 
Ruleset is empty. All predictions it makes with method .predict will be negative. It may be untrain

Accuracy after truth‑table distillation: 0.6580


### 5 · Accuracy comparison

In [6]:
pd.DataFrame({ 'Model':['Baseline LUT','Distilled RIPPER'], 'Accuracy':[acc_lut, acc_ripped] })


,Model,Accuracy
0,Baseline LUT,0.7445
1,Distilled RIPPER,0.6580


### 6 · Discussion
Does accuracy stay identical (expected) or diverge? Any rule‑compression gains?